In [ ]:
import cv2, geopandas, geojson, io, json, os, rasterio
import numpy as np
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from azure.storage.blob import BlobServiceClient
from msrest.authentication import ApiKeyCredentials
from PIL import Image
from rasterio.windows import Window

### Environment Variables

In [ ]:
custom_vision_training_key = '<enter custom vision training key>' # Move to .env
ENDPOINT = "https://<enter project endpoint>.cognitiveservices.azure.com/"

shape_file_name = 'Plot1_Labels.shp'
geojson_file_name = '{0}.geojson'.format(shape_file_name.split('.')[0])
raster_file_name = 'Plot1_RGB.tif'

data_folder = os.path.join(os.getcwd(), 'data')
golden_tree_folder = os.path.join(data_folder, 'Golden_Tree')
geojson_file_path = os.path.join(golden_tree_folder, geojson_file_name)
shape_file_path = os.path.join(golden_tree_folder, shape_file_name)
raster_path = os.path.join(golden_tree_folder, raster_file_name)

### Connect to Custom Vision Training client

In [ ]:
credentials = ApiKeyCredentials(in_headers={"Training-key": custom_vision_training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

### Manually select a Project

In [ ]:
projects = trainer.get_projects()

for project in projects:
    print(project.id)
    print(project.name)

In [ ]:
project = projects[0]
print(project.id)
print(project.name)

### Convert Shapefile to GeoJson

In [ ]:
shape_file = geopandas.read_file(shape_file_path)
shape_file.to_file(geojson_file_path, driver='GeoJSON')

### Open raster

In [ ]:
dataset = rasterio.open(raster_path)
print(dataset.width)
print(dataset.height)
print(dataset.count)

In [ ]:
labels = {}

tags = trainer.get_tags(project.id)

for tag in tags:
    print(tag.name)
    labels[tag.name] = tag

In [ ]:
# Load labels...
with open(geojson_file_path) as f:
    data = json.load(f)
    features = data['features']

count = 0

# For each label...
for feature in features:
    if feature['type'] == 'Feature' and feature['geometry']['type'] == 'Polygon':
        # Get the name of the label...
        label = feature['properties']['Class']
        print(label)

        tag = None

        # If we can't reference the tag locally...
        if labels.get(label, None) == None:           
            # Create it...
            tag = trainer.create_tag(project.id, label)
            labels[label] = tag

        tag = labels.get(label)
        
        # Create bounding box...
        coordinates = feature['geometry']['coordinates'][0]
        
        left = sorted(coordinates)[0][0]
        right = sorted(coordinates)[-1][0]
        top = sorted(coordinates, key=lambda coordinate: coordinate[1], reverse=True)[0][1]
        bottom = sorted(coordinates, key=lambda coordinate: coordinate[1], reverse=True)[-1][1]
        
        top_x, top_y = dataset.index(left, top)
        bottom_x, bottom_y = dataset.index(right, bottom)
        
        width = bottom_y - top_y
        height = bottom_x - top_x
        
        window = dataset.read(indexes=[1, 2, 3], window=rasterio.windows.Window(top_y, top_x, width, height))
        
        # Save out the window...
        profile = {
            "driver": "JPEG",
            "count": 3,
            "height": height,
            "width": width,
            "dtype": "uint8"
        }

        chip_name = 'Window_{0}.JPG'.format(count)
        
        window_path = os.path.join(golden_tree_folder, chip_name)

        with rasterio.open(window_path, 'w', **profile) as out:
            out.write(window)

        # Upload to the window to Custom Vision...
        buffer = io.BytesIO()

        im = Image.open(window_path)
        im = im.convert('RGB')
        im.save(buffer, format='JPEG')

        images = []

        images.append(ImageFileCreateEntry(name=chip_name, contents=buffer.getvalue(), tag_ids=[tag.id]))

        print('Uploading to Classification Project...')
        result = trainer.create_images_from_files(project.id, batch=ImageFileCreateBatch(images=images))

        if not result.is_batch_successful:
            for image_create_result in result.images:
                print("Image status: ", image_create_result.status)

        os.remove(window_path)
        
        count = count + 1

In [ ]:
dataset.close()